# 1. Introduction <a name="Intro"></a>

## 1.a Background Information


Gentrification is a common phenomenom happening in big and small cities. The question is: when/how is a neighbourhood starting its gentrification process? What attracts the newcomers ? If we evaluate a neighbourhood by its rent price average and its current local businesses, then compare it to the similar and trendier neighbourhoods of that same city, could we predict what type of upcoming business should be the next one to estabish itself, in order to kickstart the gentrification? In that way, could we help local councils choose which development project to choose, in order improve the life quality of their citizens?


## 1.b Problem statement

In this report, we will pick a neighbourhood of Cologne, Germany called Nippes, which is rather residential. We will put it against the other similar neighbourhoods of that same city with higher rent prices, then analyze what type of businesses would be hypothetically the next one to open up. 

## 1.c Target audience for this report

The information gathered from the web and its analysis will be a good basis to make an informed decision, when it comes to forecast what kind of business would be the best fit to improve a specific neighbourhood. In that case, entrepreneurs looking for inspiration, businesses looking to expand, or even city council members could welcome this report as valuable study.  

# 2. Data  <a name="data"></a>

## 2.a Description of the Data
We will base our research with the following data : 

1. __Average price per m² per postal code in Cologne :__ This data is available through web scraping from this webpage : https://www.miet-check.de/mietpreise/plz/koeln/2243/ . 

2. __Name of neighbourhoods per postal code :__ This data is available through web scraping from this webpage : https://www.postleitzahlen.de/plz/koeln 

3. __Venues in all different neighbourhoods of Cologne :__ This information is gathered through Geocoder and Foursquare. Once we attach the proper GPS coordinates to each neighbourhood, we'll be able gather a maximum of 75 of businesses in a radius of 500m form that location.


## 2.b How will the data be used to solve the problem

1. __Processing of Data :__ We'll be gathering the data into panda dataframes from different web sources mentioned in __2.a__
2. __Clean dataset :__ If we are missing information such as rent prices for a specific postal code, we'll be using the average of that same neighbourhood from other postal codes. Then we'll be removing any NaN, should there be any left, and making sure that the relevant info stays into the datasets
3. __Exploratory data analysis :__ 
We will be using : 
    1. Graphs to look at the different rent prices 
    4. One hot encoding and K-Means will help us cluster the similar neighbourhoods into groups.
    
## 2.c Librairies
Here are the folloring librairies which will be useful for our analysis:

In [1]:
# Main Data Science Librairies
import requests 
import pandas as pd 
import numpy as np 
import random 
import itertools

#Graphs
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import folium

# Geocoder to get the GPS coordinates
from geopy.geocoders import Nominatim
import geocoder

# Display of images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# Json Files
from pandas.io.json import json_normalize

#K-means
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


## 2.d Data preparation
### 1) Get all postal codes of Cologne and their respective neighbourhood 

It was not possible to get the table on the website as a dataframe. Something with an error 503. So we saved the info in a csv file and uploaded into GitHub. 

In [2]:
url = 'https://github.com/simcht/Coursera_Capstone/blob/main/PostalCodeCologne.csv'
cologne_postalcode = pd.read_html(url)[0]
cologne_postalcode.head()

,Unnamed: 0,Postal_code,Neighbourhood
0,NaN,50667,Altstadt-Nord
1,NaN,50668,Altstadt-Nord
2,NaN,50670,Altstadt-Nord
3,NaN,50672,Altstadt-Nord
4,NaN,50733,Nippes


Let's clean the dataframe

In [3]:
cologne_postalcode = cologne_postalcode.drop('Unnamed: 0', axis=1)

cologne_postalcode["Postal_code"] = cologne_postalcode["Postal_code"].astype(int) 

cologne_postalcode.head()

,Postal_code,Neighbourhood
0,50667,Altstadt-Nord
1,50668,Altstadt-Nord
2,50670,Altstadt-Nord
3,50672,Altstadt-Nord
4,50733,Nippes


Let's merge all postal code duplicates so we can work per postal code zones.

In [4]:
cologne_postalcode = cologne_postalcode.groupby(['Postal_code'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()

cologne_postalcode.head()

,Postal_code,Neighbourhood
0,50667,"Altstadt-Nord,Altstadt-Süd"
1,50668,"Altstadt-Nord,Neustadt-Nord"
2,50670,"Altstadt-Nord,Neustadt-Nord"
3,50672,"Altstadt-Nord,Neustadt-Nord"
4,50674,"Neustadt-Süd,Neustadt-Nord,Altstadt-Süd"


### 2) Get all rent prices per postal code

Let's upload the table with the rents.

In [5]:
url = 'https://www.miet-check.de/mietpreise/plz/koeln/2243/'
cologne_rent = pd.read_html(url)[0]
cologne_rent.head()

,#,PLZ,Mietpreis pro m2,Anzahl Einträge,Informationen
0,1,50667,15.94 Euro,185,mehr Infos
1,2,50668,17.29 Euro,405,mehr Infos
2,3,50670,16.47 Euro,428,mehr Infos
3,4,50672,18.42 Euro,393,mehr Infos
4,5,50674,16.92 Euro,395,mehr Infos


Let's clean the dataframe.

In [6]:
cologne_rent.drop(["#", "Anzahl Einträge", "Informationen"],  axis = 1, inplace = True)

cologne_rent.columns = ['Postal_code', 'Rent']

cologne_rent['Rent'] = cologne_rent['Rent'].str.replace(' Euro', '')

cologne_rent["Postal_code"] = cologne_rent["Postal_code"].astype(int) 

cologne_rent['Rent'].astype(float)

cologne_rent.head()

,Postal_code,Rent
0,50667,15.94
1,50668,17.29
2,50670,16.47
3,50672,18.42
4,50674,16.92


In [7]:
print("There are {} different postal codes and {} prices".format(cologne_rent.Postal_code.nunique(), cologne_rent.shape[0]))

There are 45 different postal codes and 45 prices


### 3) Merge both dataframes together

In [8]:
#Merge postal code with rent dataframes
cologne_merged = cologne_postalcode.merge(cologne_rent, on=["Postal_code"])
cologne_merged.sort_values(by='Neighbourhood', inplace= True)
cologne_merged.reset_index(inplace = True)
cologne_merged.drop(["index"],  axis = 1, inplace = True)
cologne_merged.head(10)

,Postal_code,Neighbourhood,Rent
0,50667,"Altstadt-Nord,Altstadt-Süd",15.94
1,50668,"Altstadt-Nord,Neustadt-Nord",17.29
2,50670,"Altstadt-Nord,Neustadt-Nord",16.47
3,50672,"Altstadt-Nord,Neustadt-Nord",18.42
4,50676,Altstadt-Süd,16.02
5,51067,"Dellbrück,Buchheim,Holweide",10.91
6,51069,"Dünnwald,Dellbrück,Höhenhaus",10.39
7,50931,Lindenthal,16.36
8,50739,"Mauenheim,Weidenpesch,Bilderstöckchen,Longerich",11.48
9,50769,"Merkenich,Worringen,Roggendorf/Thenhoven,Seebe...",9.84


Let's check whether we have missing data anywhere. If this sums up to 0, it means all cells have information.

In [9]:
cologne_merged.isnull().sum(axis = 0)

Postal_code      0
Neighbourhood    0
Rent             0
dtype: int64

### 4) Gather GPS coordinates for each neighbourhood

In [10]:
##Define function that returns latitude and longitude 
def get_cordinates(postal_code, neighbourhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, {}'.format(postal_code, neighbourhood))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude, longitude


#Loop through dataframe
for rownr in range(0,len(cologne_merged)):
    postal_code = cologne_merged.loc[rownr, 'Postal_code'] 
    neighbourhood = cologne_merged.loc[rownr, 'Neighbourhood'] 
    latitude, longitude = get_cordinates(postal_code, neighbourhood)
    cologne_merged.loc[cologne_merged.index[rownr], 'Latitude'] = latitude
    cologne_merged.loc[cologne_merged.index[rownr], 'Longitude'] = longitude

In [12]:
cologne_merged

,Postal_code,Neighbourhood,Rent,Latitude,Longitude
0,50667,"Altstadt-Nord,Altstadt-Süd",15.94,50.938611,6.955800
1,50668,"Altstadt-Nord,Neustadt-Nord",17.29,50.954208,6.966524
2,50670,"Altstadt-Nord,Neustadt-Nord",16.47,50.951869,6.948861
3,50672,"Altstadt-Nord,Neustadt-Nord",18.42,50.943656,6.934708
4,50676,Altstadt-Süd,16.02,50.931355,6.951499
5,51067,"Dellbrück,Buchheim,Holweide",10.91,50.974079,7.057209
6,51069,"Dünnwald,Dellbrück,Höhenhaus",10.39,50.994350,7.032296
7,50931,Lindenthal,16.36,50.932370,6.925441
8,50739,"Mauenheim,Weidenpesch,Bilderstöckchen,Longerich",11.48,50.984540,6.922354
9,50769,"Merkenich,Worringen,Roggendorf/Thenhoven,Seebe...",9.84,51.048691,6.919731


### 5) Request venues information via FourSquare 


Defaut information such as client ID, client Secret, Version, Limit and Radius

In [13]:
CLIENT_ID = 'XVKYTNJ0UZ1ZANJU5TMXMV0TGX4MNKV3TUD0E5Y2JB5FCVOQ' # Foursquare ID
CLIENT_SECRET = 'R5PGTISFZ5KZLAYHUFU3GLWPR1IPJ50YKKTH54UBCWPYPN1W' # Foursquare Secret
VERSION = '20201010' # Foursquare API version
LIMIT = 75 # limit of number of venues returned by Foursquare API
radius = 750 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XVKYTNJ0UZ1ZANJU5TMXMV0TGX4MNKV3TUD0E5Y2JB5FCVOQ
CLIENT_SECRET:R5PGTISFZ5KZLAYHUFU3GLWPR1IPJ50YKKTH54UBCWPYPN1W


Let's define the function that will gather the venues per neighbourhood

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
cologne_venues = getNearbyVenues(names=cologne_merged['Neighbourhood'],
                                   latitudes=cologne_merged['Latitude'],
                                   longitudes=cologne_merged['Longitude']
                                   )

In [16]:
cologne_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Altstadt-Nord,Altstadt-Süd",50.938611,6.9558,KOLUMBA Kunstmuseum,50.938530,6.954084,Art Museum
1,"Altstadt-Nord,Altstadt-Süd",50.938611,6.9558,Sattgrün,50.938441,6.954965,Vegetarian / Vegan Restaurant
2,"Altstadt-Nord,Altstadt-Süd",50.938611,6.9558,LEGO Store,50.937042,6.956564,Toy / Game Store
3,"Altstadt-Nord,Altstadt-Süd",50.938611,6.9558,Espresso Perfetto,50.938852,6.953973,Coffee Shop
4,"Altstadt-Nord,Altstadt-Süd",50.938611,6.9558,Apple Schildergasse,50.936552,6.955877,Electronics Store


In [17]:
cologne_venues.shape

(663, 7)

663 different venues found, split into their respective neighbourhood

We have gathered all necessary data into 2 different dataframes: 
1. __cologne_merged:__ Contains every postal codes, names, rent prices and GPS coordinates of every neighbourhoods of Cologne

2. __cologne_venues:__ Contains every venues with a 500-meter radius from the initial GPS coordinates of each neighbourhood, to a maxium of 75 venues. 

Next step, since our data is ready to be analyze, we'll proceed to the

__Exploratory data analysis :__ 

We will be using : 
1. Graphs to look at the different rent prices 
2. One hot encoding and K-Means will help us cluster the similar neighbourhoods into groups.